In [1]:
import numpy as np
import random
import os 
import torch
import torch.nn as nn
from torch import Tensor
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, ConcatDataset
import torch.nn.functional as F
from torch.cuda.amp.grad_scaler import GradScaler
from torch.cuda.amp import autocast

from torchsummary import summary
from sklearn.metrics import r2_score, precision_score, f1_score

from ray import tune

import json
import itertools
from itertools import groupby
import gzip 
from io import BytesIO
from time import time 

import matplotlib.pyplot as plt

import pyBigWig
from scipy.sparse import csc_matrix
import math 
from copy import deepcopy


In [2]:
targets_memmap_data_dir = '/data/users/goodarzilab/darya/work/Datasets/basenji_tutorial_targets_memmaps'
targets_memmap_data_dir_cl = '/data/users/goodarzilab/darya/work/Datasets/hg38_targets_memmaps_CL.ATAC'
targets_memmap_data_dir_pdx = '/data/users/goodarzilab/darya/work/Datasets/hg38_targets_memmaps_PDX.ATAC'
memmap_data_contigs_dir = '/data/users/goodarzilab/darya/work/Datasets/hg19_memmaps'

In [3]:
input_files_dir = memmap_data_contigs_dir
target_files_dir = [targets_memmap_data_dir_cl, targets_memmap_data_dir_pdx]
# target_files_dir = targets_memmap_data_dir

mode = 'regression'
cut = 0.8

In [5]:
chroms_list = [file.split('_')[0] for file in os.listdir(input_files_dir) if file.split('.')[-1] == 'dta']
# shuffle the files
np.random.shuffle(chroms_list)

input_list = np.hstack([[file for file in os.listdir(input_files_dir) if file.split('_')[0] == chrom] for chrom in chroms_list])


if not isinstance(target_files_dir, list): 
    target_files_dir = [target_files_dir]

targets_list = np.array([np.hstack([[file for file in os.listdir(target_dir) if file.split('_')[0] == chrom] for chrom in chroms_list]) for target_dir in target_files_dir])
val_input_files = [os.path.join(input_files_dir, file) for file in input_list[int(len(input_list)*cut):]]
train_input_files = [os.path.join(input_files_dir, file) for file in input_list[:int(len(input_list)*cut)]]

val_target_files = np.array([[os.path.join(target_files_dir[i], target_file) for target_file in targets_list[i][int(targets_list.shape[-1]*cut):]] for i in range(targets_list.shape[0])]).T
train_target_files = np.array([[os.path.join(target_files_dir[i], target_file) for target_file in targets_list[i][:int(targets_list.shape[-1]*cut)]] for i in range(targets_list.shape[0])]).T

In [6]:
def read_memmap_input(mmap_name):
    '''
    Loads a memory map input
    '''
    seq = np.memmap(mmap_name, dtype='float32',  mode = 'r+') #, shape=(2, self.chrom_seq[self.chrom]))
    return seq


In [7]:
tgts = [read_memmap_input(mmap_name) for mmap_name in train_target_files[0]]

In [8]:
seq = read_memmap_input(train_input_files[0])

In [9]:
num_targets_lst = [2, 3]

In [10]:
num_targets_lst.max()

AttributeError: 'list' object has no attribute 'max'

In [24]:
def get_means(tgt, ids):
    val = torch.mean(torch.tensor(np.nan_to_num(np.take(tgt, ids))), dim=1)
    return val

In [41]:
max_target_len = 3
chrom_len = seq.shape[0]
target_window = 128
idx = 0


arr = np.arange(idx*target_window, (idx+1) * target_window)
tgt_lst = np.arange(0, chrom_len*max_target_len, chrom_len)
ids = np.array([np.split(arr, 128) + tgt for tgt in tgt_lst])
stacked_means = torch.cat([get_means(tgts[i], ids[:num_targets_lst[i]]) for i in range(len(tgts))])

In [42]:
vals = [get_means(tgts[i], ids[:num_targets_lst[i]]) for i in range(len(tgts))]

In [43]:
vals[0]

tensor([[0.0000],
        [0.0045]])

In [45]:
torch.flip(vals[0], dims=[0, 1])

tensor([[0.0045],
        [0.0000]])

In [ ]:
# stacked_means = torch.mean(torch.tensor(np.nan_to_num(np.take(tgt_mmap_cl, ids_cl))), dim=1)
# stacked_means_pdx = torch.mean(torch.tensor(np.nan_to_num(np.take(self.tgt_mmap_pdx, ids_pdx))), dim=1)
# tgt_window = torch.cat((stacked_means_cl, stacked_means_pdx), dim=0)


In [ ]:
train_target_files[0]

In [48]:
training_dset = ConcatDataset([Main_Dataset(train_input_files[i], 
                                        train_target_files[i], [2, 3],
                                        128*16,
                                        switch=False) 
                               for i in range(len(train_input_files))])


In [49]:
batch_size = int(128*128*8 *4/ (128*16))

In [50]:
train_loader = DataLoader(dataset=training_dset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=False)

In [51]:
start = time()
batch = next(iter(train_loader))
print (time() - start)

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f1cdb0e3280>
Traceback (most recent call last):
  File "/data/users/goodarzilab/darya/anaconda3/envs/basenji_pytorch/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/data/users/goodarzilab/darya/anaconda3/envs/basenji_pytorch/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/data/users/goodarzilab/darya/anaconda3/envs/basenji_pytorch/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f1cdb0e3280>
Traceback (most recent call last):
  File "/data/users/goodarzilab/darya/anaconda3/envs/basenji_pytorch/lib/python3.9/site-packages/torch/utils/data/dataloader.py", 

23.597992181777954


In [100]:
# if isinstance(target_files_dir, list): 
#     targets_list = np.array([np.hstack([[file for file in os.listdir(target_dir) if file.split('_')[0] == chrom] for chrom in chroms_list]) for target_dir in target_files_dir])
# else: 
#     targets_list = np.expand_dims(np.hstack([[file for file in os.listdir(target_files_dir) if file.split('_')[0] == chrom] for chrom in chroms_list]), axis=0)

In [47]:
class Main_Dataset(Dataset):
    '''
    A Dataset class. 
    For each chromosome, opens the input and target file, 
    with each __getitem__ call, returns one-hot encoded input and target averaged out over a given window - default is 128. 
    Attributes:         
        target_window (int) - size of the slice from the input and target arrays. Default is 128. 
        seq (mmap) - input file
        tgts_mmap_cl (lst of mmaps) - a list of memory maps corresponding to each target in the target lst 
        chrom_len (int) - length of the selected chromosome
        nucs (arr) - nucleotides encoded to ints, N -> 5
        switch (bool) - if True, the nucleotide sequence gets reversed 
        switch_func (func) - vectorized function to reverse the nuc sequence 
        num_targets_lst (int) - a list with the number(s) of targets 
        

'''
    def __init__(self, input_name, targets_name, num_targets, target_window, switch=False):

        self.target_window = target_window
        self.seq = self.read_memmap_input(input_name)
        self.tgts = [self.read_memmap_input(mmap_name) 
                for mmap_name in targets_name]
#         self.tgt_mmap_cl = self.read_memmap_input(targets_name_cl)
#         self.tgt_mmap_pdx = self.read_memmap_input(targets_name_pdx)
        self.chrom_len = self.seq.shape[0]
        self.nucs = np.arange(6.)
        self.switch = switch 
        self.switch_func = np.vectorize(lambda x: x + 1 if (x % 2 == 0) else x - 1)
#         self.num_targets_cl = 2 #23
#         self.num_targets_pdx = 2 #14
        if not isinstance(num_targets, list):
            num_targets = [num_targets]
        self.num_targets_lst = num_targets
        
    def __len__(self):
        # length of the dataset is defined as the ration between the full length of the input and the target window
        return int(self.seq.shape[0] / self.target_window)

    def __getitem__(self, idx): 
        # slice the input from the memory map
        seq_subset = self.seq[idx*self.target_window:(idx+1)*self.target_window]
        # if switch=True, reverse the nuc sequence
        if self.switch: 
            seq_subset = self.switch_func(list(reversed(seq_subset)))
        # one-hot encode the input, here compressed row format -> sparse matrix conversion is used
        dta = self.get_csc_matrix(seq_subset)
        max_target_len = max(num_targets_lst)
        arr = np.arange(idx*self.target_window, (idx+1) * self.target_window)
        tgt_lst = np.arange(0, self.chrom_len*max_target_len, self.chrom_len)
        ids = np.array([np.split(arr, 128) + tgt for tgt in tgt_lst])
#         print (self.num_targets_lst, [i for i in range(len(self.tgts))])
#         print ([self.num_targets_lst[i] for i in range(len(self.tgts))])
        stacked_targets_means = torch.cat([self.get_means(self.tgts[i], ids[:self.num_targets_lst[i]]) for i in range(len(self.tgts))])
        return torch.tensor(dta), stacked_targets_means 

    def read_memmap_input(self, mmap_name):
        '''
        Loads a memory map input
        '''
        seq = np.memmap(mmap_name, dtype='float32',  mode = 'r+') #, shape=(2, self.chrom_seq[self.chrom]))
        return seq

    def get_csc_matrix(self, seq_subset):
        '''
        Converts a compressed row format data to a sparse matrix
        ''' 
        N, M = len(seq_subset), len(self.nucs)
        rows, cols = np.arange(N), seq_subset
        data = np.ones(N, dtype=np.uint8)
        ynew = csc_matrix((data, (rows, cols)), shape=(N, M), dtype=np.uint8)
        return ynew.toarray()[:, :4]
    
    def get_means(self, tgt, ids):
        val = torch.mean(torch.tensor(np.nan_to_num(np.take(tgt, ids))), dim=1)
        if self.switch: 
            vals = torch.flip(vals, dims=[0, 1])
#         print (val.shape)
        return val

In [ ]:
def make_dsets(self, input_files_dir, target_files_dir, num_targets, mode):
    cut = self.param_vals.get('cut', .8)
    np.random.seed(42)
    chroms_list = [file.split('_')[0] for file in os.listdir(input_files_dir) if file.split('.')[-1] == 'dta']
    # shuffle the files
    np.random.shuffle(chroms_list)
    input_list = np.hstack([[file for file in os.listdir(input_files_dir) if file.split('_')[0] == chrom] for chrom in chroms_list])


    if not isinstance(target_files_dir, list): 
        target_files_dir = [target_files_dir]

    targets_list = np.array([np.hstack([[file for file in os.listdir(target_dir) if file.split('_')[0] == chrom] for chrom in chroms_list]) for target_dir in target_files_dir])
    val_input_files = [os.path.join(input_files_dir, file) for file in input_list[int(len(input_list)*cut):]]
    train_input_files = [os.path.join(input_files_dir, file) for file in input_list[:int(len(input_list)*cut)]]

    val_target_files = np.array([[os.path.join(target_files_dir[i], target_file) for target_file in targets_list[i][int(targets_list.shape[-1]*cut):]] for i in range(targets_list.shape[0])]).T
    train_target_files = np.array([[os.path.join(target_files_dir[i], target_file) for target_file in targets_list[i][:int(targets_list.shape[-1]*cut)]] for i in range(targets_list.shape[0])]).T
    
    if self.mode=='classification': 
        self.valid_dset = ConcatDataset([Toy_Dataset(os.path.join(input_files_dir, val_input_files[i]),
                                                         self.param_vals.get('target_window', 128)
                                                        ) for i in range(len(val_input_files))])

        self.training_dset = ConcatDataset([Toy_Dataset(os.path.join(input_files_dir, train_input_files[i]), 
                                            self.param_vals.get('target_window', 128),
                                                     switch=False) for i in range(len(train_input_files))])

        self.training_dset_augm = ConcatDataset([Toy_Dataset(os.path.join(input_files_dir, train_input_files[i]),
                                                             self.param_vals.get('target_window', 128),
                                                          switch=True) for i in range(len(train_input_files))])
    else: 

        self.valid_dset = ConcatDataset([Main_Dataset(val_input_files[i], 
                                    val_target_files[i], num_targets,
                                    self.param_vals.get('target_window', 128),
                                    switch=False) 
                           for i in range(len(train_input_files))])

        self.training_dset = ConcatDataset([Main_Dataset(train_input_files[i], 
                        train_target_files[i], num_targets,
                        self.param_vals.get('target_window', 128),
                        switch=False) 
               for i in range(len(train_input_files))])

        self.training_dset_augm = ConcatDataset([Main_Dataset(train_input_files[i], 
                                    train_target_files[i], num_targets,
                                    self.param_vals.get('target_window', 128),
                                    switch=False) 
                           for i in range(len(train_input_files))])


In [ ]:
#     def make_dsets(input_files_dir, target_files_dir, target_files_dir, mode):
#         '''
#         Initializes the datasets
#         '''
#         cut = 0.8 #self.param_vals.get('cut', .8)
#         np.random.seed(42)
#         # select the files with the .dta extension
#         chroms_list = [file.split('_')[0] for file in os.listdir(input_files_dir) if file.split('.')[-1] == 'dta']
#         # shuffle the files
#         np.random.shuffle(chroms_list)
#         # create the input and target file lists for training and validation datasets 
#         input_list = np.hstack([[file for file in os.listdir(input_files_dir) if file.split('_')[0] == chrom] for chrom in chroms_list])
#         targets_cl_list = np.hstack([[file for file in os.listdir(target_files_dir_cl) if file.split('_')[0] == chrom] for chrom in chroms_list])
#         targets_pdx_list = np.hstack([[file for file in os.listdir(target_files_dir_pdx) if file.split('_')[0] == chrom] for chrom in chroms_list])
        
#         val_input_files = input_list[int(len(input_list)*cut):]
#         val_target_files_cl = targets_cl_list[int(len(targets_cl_list)*cut):]
#         val_target_files_pdx = targets_pdx_list[int(len(targets_pdx_list)*cut):]

        
#         train_input_files = input_list[:int(len(input_list)*cut)]
#         train_target_cl_files = targets_cl_list[:int(len(targets_cl_list)*cut)]
#         train_target_pdx_files = targets_pdx_list[:int(len(targets_pdx_list)*cut)]

#         # concatenate the datasets defined for each chromosome 
#         if self.mode=='classification': 
#             self.valid_dset = ConcatDataset([Toy_Dataset(os.path.join(input_files_dir, val_input_files[i]),
#                                                              self.param_vals.get('target_window', 128)
#                                                             ) for i in range(len(val_input_files))])
        
#             self.training_dset = ConcatDataset([Toy_Dataset(os.path.join(input_files_dir, train_input_files[i]), 
#                                                 self.param_vals.get('target_window', 128),
#                                                          switch=False) for i in range(len(train_input_files))])

#             self.training_dset_augm = ConcatDataset([Toy_Dataset(os.path.join(input_files_dir, train_input_files[i]),
#                                                                  self.param_vals.get('target_window', 128),
#                                                               switch=True) for i in range(len(train_input_files))])
#         else: 
#             self.valid_dset = ConcatDataset([DNA_Iter(os.path.join(input_files_dir, val_input_files[i]), 
#                                                       os.path.join(target_files_dir_cl, val_target_files_cl[i]), 
#                                                       os.path.join(target_files_dir_pdx, val_target_files_pdx[i]), 
#                                                       self.param_vals.get('target_window', 128)
#                                                       ) for i in range(len(val_input_files))])

#             self.training_dset = ConcatDataset([DNA_Iter(os.path.join(input_files_dir, train_input_files[i]), 
#                                                          os.path.join(target_files_dir_cl, train_target_cl_files[i]), 
#                                                          os.path.join(target_files_dir_pdx, train_target_pdx_files[i]), 
#                                                          self.param_vals.get('target_window', 128),
#                                                          switch=False) for i in range(len(train_input_files))])

#             self.training_dset_augm = ConcatDataset([DNA_Iter(os.path.join(input_files_dir, train_input_files[i]), 
#                                                               os.path.join(target_files_dir_cl, train_target_cl_files[i]),  
#                                                               os.path.join(target_files_dir_pdx, train_target_pdx_files[i]), 
#                                                               self.param_vals.get('target_window', 128),
#                                                               switch=True) for i in range(len(train_input_files))])
